<a href="https://www.nvidia.com/dli"> <img src="images/combined_logo.png" alt="Header" style="width: 400px;"/> </a>

# Disaster Risk Monitoring Using Satellite Imagery #

## 00 - Introduction ##
Welcome to the NVIDIA DLI _Disaster Risk Monitoring Using Satellite Imagery_ lab. In this lab, you will learn how to build and deploy a flood detection segmentation model using data collected from Sentinel-1 satellites. In order to achieve this, we will use NVIDIA's _DALI_, _TAO Toolkit_, _TensorRT_, and _Triton Inference Server_. 

**Learning Objectives**
<br>
In this course, you will learn important concepts about using satellite imagery to develop deep learning-based disaster risk monitoring systems, including: 
* The Application of Computer Vision for Disaster Risk Monitoring
* Manipulation of Data Collected by Earth Observation Satellites 
* Ways to Efficiently Process Large Imagery Data
* Deep Learning Model Development Challenges
* End-to-End Machine Learning Workflow

**Table of Contents**
<br>
This lab is broken down into four parts. In the first part, our focus is primarily on learning how to process large satellite images. In the second part of the lab, we will learn to efficiently train a segmentation model. Next, we will deploy the model in the third part of the lab. In the last part of the lab, we will utilize the deep learning model to perform an impact analysis. This hands-on lab will provide opportunities to get familiar with each step in the development workflow and to enable customizations that are most appropriate for your own use cases. 
1. Disaster Risk Monitoring Systems and Data Pre-processing
2. Efficient Model Training
3. Model Deployment for Inference
4. UNOSAT Flood Event Case Study

### JupyterLab ###
For this hands-on lab, we use [JupyterLab](https://jupyterlab.readthedocs.io/en/stable/) to manage our environment.  The [JupyterLab Interface](https://jupyterlab.readthedocs.io/en/stable/user/interface.html) is a dashboard that provides access to interactive iPython notebooks, as well as the folder structure of our environment and a terminal window into the Ubuntu operating system. The first view you will see includes a **menu bar** at the top, a **file browser** in the **left sidebar**, and a **main work area** that is initially open to this "introduction" notebook. 
<p><img src="images/jl_launcher.png"></p>

* The file browser can be navigated just like any other file explorer. A double click on any of the items will open a new tab with its content. 
* The main work area includes tabbed views of open files that can be closed, moved, and edited as needed. 
* The notebooks, including this one, consist of a series of content and code **cells**. To execute code in a code cell, press `Shift+Enter` or the `Run` button in the menu bar above, while a cell is highlighted. Sometimes, a content cell will get switched to editing mode. Executing the cell with `Shift+Enter` or the `Run` button will switch it back to a readable form.
* To interrupt cell execution, click the `Stop` button in the menu bar or navigate to the `Kernel` menu, and select `Interrupt Kernel`. 
* We can use terminal commands in the notebook cells by prepending an exclamation point/bang(`!`) to the beginning of the command. 

<a name='e1'></a>
#### Exercise #1 - Practice ####
Try executing the simple print statement in the cell below.

In [1]:
# activate this cell by selecting it with the mouse or arrow keys then use the keyboard shortcut [Shift+Enter] to execute
print('This is just a simple print statement')

This is just a simple print statement


Now try executing the terminal command in the cell below.

In [2]:
# activate this cell by selecting it with the mouse or arrow keys then use the keyboard shortcut [Shift+Enter] to execute
!echo 'This is another simple print statement'

This is another simple print statement


### NVIDIA GPU Cloud ###
This lab requires that the user have an NVIDIA GPU Cloud (NGC) account and API key.  If you have not done so already, please: 
1. [Register and activate a free NGC account](https://docs.nvidia.com/ngc/ngc-overview/index.html#registering-activating-ngc-account)
2. [Generate your NGC API key](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) and save it in a safe location

<a name='e2'></a>
#### Exercise #2 - NGC CLI and Docker Registry ####

**Instructions**: <br>
* Edit the cell below by replacing `<MY_NGC_API_KEY>` with your key and execute the cell to set the variable. 
* Execute the cell below to create a configuration file for NGC. 
* Execute the cell below to move the newly created config file to `.ngc` to configure the NGC CLI. 
* Execute the cell below to log in to NGC's Docker registry.
* Execute the cell below to pull the TAO container images.

In [3]:
apikey='ZmcwM3IxYXRqaThpdmZibWI1a2k4cjkwc2k6MDJjZGIyMDctY2FiMS00MTYyLWI4YzAtNzI1YmVmZDZhODli'

In [4]:
# write the config file
config_dict={'apikey': apikey, 'format_type': 'json', 'org': 'nvidia'}
with open('config', 'w') as f: 
    f.write(';WARNING - This is a machine generated file.  Do not edit manually.\n')
    f.write(';WARNING - To update local config settings, see "ngc config set -h"\n')
    f.write('\n[CURRENT]\n')
    for k, v in config_dict.items(): 
        f.write(k+'='+v+'\n')

# preview the config file
!cat config

;WARNING - This is a machine generated file.  Do not edit manually.
;WARNING - To update local config settings, see "ngc config set -h"

[CURRENT]
apikey=ZmcwM3IxYXRqaThpdmZibWI1a2k4cjkwc2k6MDJjZGIyMDctY2FiMS00MTYyLWI4YzAtNzI1YmVmZDZhODli
format_type=json
org=nvidia


In [5]:
# move the config file to ~/.ngc
!mkdir -p ~/.ngc & mv config ~/.ngc/

In [6]:
# login to NGC's docker registry
!docker login -u '$oauthtoken' -p $apikey nvcr.io

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [7]:
# pull TAO docker container images
!docker pull nvcr.io/nvidia/tao/tao-toolkit:5.0.0-tf1.15.5
!docker pull nvcr.io/nvidia/tao/tao-toolkit:5.5.0-deploy

5.0.0-tf1.15.5: Pulling from nvidia/tao/tao-toolkit

715873ec: Pulling fs layer 
5fe751be: Pulling fs layer 
c9e56778: Pulling fs layer 
9c08065c: Pulling fs layer 
129e9daa: Pulling fs layer 
ddf5daef: Pulling fs layer 
b1e8d85a: Pulling fs layer 
bd04fbf5: Pulling fs layer 
0bd8cf35: Pulling fs layer 
15e856a0: Pulling fs layer 
618a5cab: Pulling fs layer 
c8820090: Pulling fs layer 
305027fa: Pulling fs layer 
b700ef54: Pulling fs layer 
36ff9510: Pulling fs layer 
bcc98e9c: Pulling fs layer 
98675d88: Pulling fs layer 
41639381: Pulling fs layer 
aa1cb86b: Pulling fs layer 
dc4c1852: Pulling fs layer 
84394baa: Pulling fs layer 
5c59bbac: Pulling fs layer 
cf598fa6: Pulling fs layer 
2d620b14: Pulling fs layer 
ad30cb0b: Pulling fs layer 
503756df: Pulling fs layer 
ab4d6cbe: Pulling fs layer 
3f79fc24: Pulling fs layer 
458e2746: Pulling fs layer 
dd98ec7a: Pulling fs layer 
3121b2fc: Pulling fs layer 
98142162: Pulling fs layer 
3d1811c0: Pulling fs layer 
53e2569a: Pulling fs la

**Well Done!** If the output includes _"Login Succeed"_, then you are ready to begin. Let's move to the [next notebook](./01_disaster_risk_monitoring_systems_and_data_pre-processing.ipynb). 

<a href="https://www.nvidia.com/dli"> <img src="images/combined_logo.png" alt="Header" style="width: 400px;"/> </a>